This script imputes Supplemental Security Income (SSI) recipients and dollar benefit amount to match the aggregates with Social Security Administration (SSA) statistics for SSI. In this current version, we used 2015 CPS data and SSA [2014 annual reports on SSI](https://www.ssa.gov/policy/docs/statcomps/ssi_asr/). Please refer to the documentation in the same folder for more details on methodology and assumptions. The output this script is a personal level dataset that contains CPS personal level ID (PERIDNUM), individual participation indicator (SSI_participation, 0 - not a recipient, 1 - current recipient on file, 2 - imputed recipient), and benefit amount.

Input: 2015 CPS (cpsmar2015t.csv), 2014 number of recipients by age and state in Dec 2014 (SSI_by_age_state.csv)

Output: SSI_Imputation.csv

Additional Source links: https://www.ssa.gov/policy/docs/statcomps/ssi_sc/2015/table02.html

In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import random
import statsmodels.discrete.discrete_model as sm

Trim the dataset and only keep columns relevant to SSI augmentation

In [ ]:
CPS_dataset = pd.read_csv('asec2015_pubuse.csv') 

In [ ]:
columns_to_keep = ['ssi_val', 'ssi_yn','ssikidyn', 'resnssi1', 'resnssi2', 'marsupwt', 'a_age', 'gestfips',
                   'pedisdrs', 'pedisear', 'pediseye', 'pedisout', 'pedisphy', 'pedisrem',
                   'dis_hp', 'rsnnotw', 'vet_typ1', 'pemlr', 'mcare', 'a_ftpt', 'filestat',
                   'wsal_val', 'semp_val', 'frse_val', 'ss_val', 'rtm_val', 'oi_off', 'oi_val',
                   'uc_yn', 'uc_val', 'int_yn', 'int_val', 'a_spouse', 'paw_yn', 'vet_yn', 
                   'ffpos', 'fh_seq', 'finc_ssi', 'ftot_r', 'ftotval', 'ptot_r', 'ptotval',
                   'peridnum', 'h_seq', 'i_ssival', 'fwsval', 'wc_yn', 'ss_yn', 
                   'sur_yn', 'dis_yn', 'hed_yn', 'hcsp_yn', 'hfdval', 'mcaid']

CPS_dataset = CPS_dataset[columns_to_keep]
CPS_dataset.to_csv('CPSASEC_SSI.csv', columns=columns_to_keep, index=False)

In [3]:
CPS_dataset = pd.read_csv('CPSASEC_SSI.csv')

# Converting age and SSI benefit to numeric values

In [4]:
# for now assuming ages are evenly distributed in that range
# how to make sure they're 80 years old and 85 years old?
CPS_dataset.a_age = np.where(CPS_dataset.a_age == "80-84 years of age", 80,
                             CPS_dataset.a_age)
CPS_dataset.a_age = np.where(CPS_dataset.a_age == "85+ years of age", 85,
                             CPS_dataset.a_age)
CPS_dataset.a_age = pd.to_numeric(CPS_dataset.a_age)

In [5]:
# all existing ssi_val are greater than 0 (min==1)
CPS_dataset.ssi_val = np.where(CPS_dataset.ssi_val == "None or not in universe",'0', CPS_dataset.ssi_val)
CPS_dataset.ssi_val = pd.to_numeric(CPS_dataset.ssi_val)

# Create disability proxy indicator

In [6]:
a1 = (CPS_dataset.pedisdrs == 'Yes')
a2 = (CPS_dataset.pedisear == 'Yes')
a3 = (CPS_dataset.pediseye == 'Yes')
a4 = (CPS_dataset.pedisout == 'Yes')
a5 = (CPS_dataset.pedisphy == 'Yes')
a6 = (CPS_dataset.pedisrem == 'Yes')
disability = (a1|a2|a3|a4|a5|a6)

In [7]:
d1 = (CPS_dataset.dis_hp == 'Yes')
d2 = (CPS_dataset.pemlr == 'Not in labor force - disabled')
d3 = (CPS_dataset.rsnnotw == 'Ill or disabled')
d4 = (CPS_dataset.vet_typ1 == 'Yes')
d5 = (CPS_dataset.a_age < 65) & (CPS_dataset.a_age >= 18) & (CPS_dataset.mcare == 'Yes')
work_disability = (d1|d2|d3|d4|d5)

# Create SSI countable income proxy

In [8]:
def countable(earned, unearned, deemed=None):
    if deemed is not None:
        SSI_countable = earned + unearned + deemed
    else:
        SSI_countable = earned + unearned
        
    SSI_countable = np.where(SSI_countable > 0, SSI_countable, 0)
    
    # disgards
    SSI_countable = np.where(SSI_countable>20*12, SSI_countable - 20*12, 0) # exclude $20 from most income items
    SSI_countable = np.where((earned > 65*12)&(SSI_countable > 65*12), SSI_countable - 65*12, 0) # exclude he first $65 of earnings 
    SSI_countable = np.where((earned > 65*12)&(SSI_countable > 0.5*(earned - 65*12)), 
                             SSI_countable - 0.5*(earned - 65*12), 0) # one–half of earnings over $65 received in a month 
    
    return SSI_countable

In [9]:
wage = pd.to_numeric(np.where(CPS_dataset.wsal_val!= 'None or not in universe', CPS_dataset.wsal_val, 0))
self_employed1 = pd.to_numeric(np.where(CPS_dataset.semp_val!= 'None or not in universe', CPS_dataset.semp_val, 0))
self_employed2 = pd.to_numeric(np.where(CPS_dataset.frse_val!= 'None or not in universe', CPS_dataset.frse_val, 0))
earned = wage + self_employed1 + self_employed2

In [10]:
ss = pd.to_numeric(np.where(CPS_dataset.ss_val!='None or not in universe', CPS_dataset.ss_val, 0))
pension = pd.to_numeric(np.where(CPS_dataset.rtm_val!='None or not in universe', CPS_dataset.rtm_val, 0))
#too under-reported?
disability = pd.to_numeric(np.where(CPS_dataset.oi_off=='State disability payments', CPS_dataset.oi_val, 0))
unemploy = pd.to_numeric(np.where(CPS_dataset.uc_yn=='Yes', CPS_dataset.uc_val, 0))
interest = pd.to_numeric(np.where(CPS_dataset.int_yn=='Yes', CPS_dataset.int_val, 0))
unearned = ss + pension + disability + unemploy + interest

In [11]:
no_public_assistance = (CPS_dataset.paw_yn != 'Yes')
not_married_HH = (CPS_dataset.filestat=='Single')|(CPS_dataset.filestat=='Nonfiler')
student_under22 = (CPS_dataset.a_ftpt != 'Not in universe or children and')&(CPS_dataset.a_age < 22)
under_18 = (CPS_dataset.a_age < 18)
ineligible_children = np.where(no_public_assistance&(student_under22|under_18), 1, 0)

In [12]:
CPS_dataset['SSI_countable'] = countable(earned, unearned)

In [13]:
combined = np.where((disability==1)|(work_disability==1), 1, 0)
aged = np.where(CPS_dataset.a_age>=65, 1, 0)

In [14]:
subfields = {'old_disabled': (combined==1)&(aged==1),
             'young_disabled': (combined==1)&(aged==0),
             'old_not_disabled': (combined==0)&(aged==1),
             'young_not_disabled': (combined==0)&(aged==0)}

In [15]:
low_income = np.zeros(len(CPS_dataset))
for key in subfields:
    benchmark = np.percentile(CPS_dataset.SSI_countable[CPS_dataset.index[(CPS_dataset.ssi_yn=='Yes')&subfields[key]]], 95)
    low_income = np.where((CPS_dataset.SSI_countable <= benchmark) & subfields[key], 1, low_income)
    #print "group: %s, benchmark: %d" % (key, benchmark)
    print ("group: %s, benchmark: %d" % (key,benchmark)) #version of print() that works in Python 3.6

group: young_disabled, benchmark: 1609
group: old_not_disabled, benchmark: 0
group: old_disabled, benchmark: 0
group: young_not_disabled, benchmark: 18642


In [16]:
CPS_dataset['low_income'] = low_income
CPS_dataset['ineligible_children'] = ineligible_children
CPS_dataset['earned'] = earned
CPS_dataset['unearned'] = unearned
CPS_dataset['wage'] = wage

# Defining SSI target records

In [17]:
SSI_target_pool = np.zeros(len(CPS_dataset))
SSI_target_pool = np.where((low_income==1)&disability, 1, 0)
SSI_target_pool = np.where((work_disability==1)&(low_income==1), 1, SSI_target_pool)
SSI_target_pool = np.where((low_income==1)&(CPS_dataset.a_age<18)&disability, 1, SSI_target_pool)
SSI_target_pool = np.where((low_income==1)&(CPS_dataset.a_age>=65), 1, SSI_target_pool)

In [18]:
CPS_dataset['current_recipient'] = np.where(((CPS_dataset.ssi_yn =='Yes')|(CPS_dataset.ssikidyn=='Received SSI')), 1, 0)
SSI_target_pool = np.where(CPS_dataset.current_recipient==1, 1, SSI_target_pool)

In [19]:
SSI_target_pool = np.where(CPS_dataset.fwsval>57500, 0, SSI_target_pool)

In [20]:
CPS_dataset['SSI_target_pool'] = SSI_target_pool

# import administrative totals

In [21]:
admin = pd.read_csv("SSI_by_age_state.csv", 
                    dtype={'Under 18': np.float, '18-64':np.float, '65 or older': np.float,
                           'Under 18 mean': np.float, '18-64 mean':np.float, '65 or older mean': np.float})
admin.index = admin.Fips

In [22]:
# adjust targets and export adjusted targets to a CSV file
state_benefit = {}
state_recipients = {}
for state in admin.Fips:
    this_state = (CPS_dataset.gestfips==state)
    CPS_total = (CPS_dataset.ssi_val * CPS_dataset.marsupwt)[this_state].sum()/1000000
    federal_total = admin['State Total'][state]/1000
    federal_total_adj = admin['State Total'][state] * 12.096/1000
    
    state_state = admin["State-administered Supplement 2010"][state]/1000000
    state_state_adj = admin["State-administered Supplement 2010"][state] * 721/674/1000000
    admin_total = federal_total_adj + state_state_adj
    
    temp = [admin['State'][state], CPS_total, federal_total, federal_total_adj, 
            state_state, state_state_adj, admin_total]
    state_benefit[state] = temp
    
pre_augment_benefit = DataFrame(state_benefit).transpose()
pre_augment_benefit.columns = ['State', 'CPS total', 'Federal Total', 'Federal Total after Adjustment',
                               'State Total', 'State Total After adjustment', 'Admin Total']

In [23]:
pre_augment_benefit.to_csv('pre-blow-up.csv')

In [24]:
CPS_dataset.marsupwt[CPS_dataset.ssi_yn=='Yes'].sum()

6846928.109999987

In [25]:
# calculate difference of SSA stats and CPS aggregates on recipients' number
# by state and age
diff = {'Fips': [], 'Under 18':[], '18-64':[], '65 or older':[],
        'Under 18 mean':[], '18-64 mean':[], '65 or older mean':[],
       'Under 18 cps':[], '18-64 cps':[], '65 or older cps':[],
       'Under 18 ssa':[], '18-64 ssa':[], '65 or older ssa':[]}
diff['Fips'] = admin.Fips

current = (CPS_dataset.current_recipient==1)
category = {'Under 18': (CPS_dataset.a_age<18),
            '18-64': (CPS_dataset.a_age>=18)&(CPS_dataset.a_age<65),
            '65 or older': (CPS_dataset.a_age>=65)}

augment_ratio = {'Under 18': 1.401/1.299,
                 '18-64': 5.523/4.626,
                 '65 or older': 2.344/2.122}

for group in ['Under 18', '18-64', '65 or older']:
    for FIPS in admin.Fips:
        this_state = (CPS_dataset.gestfips==FIPS)
        age_range = category[group]
        current_tots = CPS_dataset.marsupwt[current&this_state&age_range].sum()
        
        valid_num = CPS_dataset.marsupwt[current&this_state&age_range].sum() + 0.0000001
        current_mean = ((CPS_dataset.ssi_val*CPS_dataset.marsupwt)[current&this_state&age_range]).sum()/valid_num
        
        this_admin = augment_ratio[group] * float(admin[group][admin.Fips==FIPS])
        diff[str(group) + ' cps'].append(current_tots)
        diff[str(group) + ' ssa'].append(float(admin[group][admin.Fips==FIPS]))
        diff[group].append(this_admin - current_tots)
        diff[str(group + ' mean')].append(float(current_mean)/12)

In [26]:
d = DataFrame(diff)
d.to_csv('recipients.csv', index=False)

In [27]:
CPS_dataset['ssi_indicator'] = np.where((CPS_dataset.ssi_yn=='Yes')|(CPS_dataset.ssikidyn=='Received SSI'), 1, 0)
CPS_dataset['paw_yn'] = np.where(CPS_dataset.paw_yn=='Yes', 1, 0)
CPS_dataset['wc_yn'] = np.where(CPS_dataset.wc_yn=='Yes', 1, 0)
CPS_dataset['uc_yn'] = np.where(CPS_dataset.uc_yn=='Yes', 1, 0)
CPS_dataset['ss_yn'] = np.where(CPS_dataset.ss_yn=='Yes', 1, 0)
CPS_dataset['sur_yn'] = np.where(CPS_dataset.sur_yn=='Yes', 1, 0)
CPS_dataset['dis_yn'] = np.where(CPS_dataset.dis_yn=='Yes', 1, 0)
CPS_dataset['vet_yn'] = np.where(CPS_dataset.vet_yn=='Yes', 1, 0)
CPS_dataset['hed_yn'] = np.where(CPS_dataset.hed_yn=='Yes', 1, 0)
CPS_dataset['hcsp_yn'] = np.where(CPS_dataset.hcsp_yn=='Yes', 1, 0)
CPS_dataset['hfdval'] = np.where(CPS_dataset.hfdval!='Not in universe', 1, 0)
CPS_dataset['countable_income'] = CPS_dataset.SSI_countable
CPS_dataset['mcare'] = np.where(CPS_dataset.mcare=='Yes', 1, 0)
CPS_dataset['mcaid'] = np.where(CPS_dataset.mcaid=='Yes', 1, 0)

SSI_disability = ((CPS_dataset.resnssi1!='NIU')&(CPS_dataset.resnssi1!='Other (adult or child)'))|((CPS_dataset.resnssi2!='NIU')&(CPS_dataset.resnssi2!='Other (adult or child)'))
CPS_dataset['combined_disability'] = np.where(disability|work_disability|SSI_disability, 1, 0)

In [28]:
CPS_dataset['age_dummy18'] = np.where(CPS_dataset.a_age<18, 1, 0)
CPS_dataset['age_dummy65'] = np.where(CPS_dataset.a_age>=65, 1, 0)

In [29]:
CPS_dataset['intercept'] = np.ones(len(CPS_dataset))
CPS_dataset['age18_X_disability'] = CPS_dataset.age_dummy18 * CPS_dataset.combined_disability
CPS_dataset['age65_X_disability'] = CPS_dataset.age_dummy65 * CPS_dataset.combined_disability
model = sm.Logit(endog= CPS_dataset.ssi_indicator, 
                 exog= CPS_dataset[['intercept','countable_income','combined_disability','uc_yn', 'ss_yn', 'wc_yn',
                                    'paw_yn', 'vet_yn', 'sur_yn', 'hed_yn', 'hcsp_yn', 'hfdval', 'mcare', 'mcaid',
                                    'age_dummy18','age_dummy65','age18_X_disability', 'age65_X_disability']])
results = model.fit()

Optimization terminated successfully.
         Current function value: 0.034985
         Iterations 12


In [30]:
#print results.summary()
print (results.summary()) #version of print() function that works in Python 3.6

                           Logit Regression Results                           
Dep. Variable:          ssi_indicator   No. Observations:               199024
Model:                          Logit   Df Residuals:                   199006
Method:                           MLE   Df Model:                           17
Date:                Mon, 28 Aug 2017   Pseudo R-squ.:                  0.6483
Time:                        11:35:32   Log-Likelihood:                -6962.9
converged:                       True   LL-Null:                       -19798.
                                        LLR p-value:                     0.000
                          coef    std err          z      P>|z|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------
intercept              -8.3085      0.111    -74.727      0.000        -8.526    -8.091
countable_income    -4.301e-05   5.07e-06     -8.485      0.000     -5.29e-05 -3.31e-05
combined_disabil

In [31]:
probs = results.predict()
CPS_dataset['probs'] = probs

In [32]:
CPS_dataset['impute'] = np.zeros(len(CPS_dataset))
CPS_dataset['ssi_impute'] = np.zeros(len(CPS_dataset))

non_current = (CPS_dataset.current_recipient == 0)
current = (CPS_dataset.current_recipient == 1)
targets = (CPS_dataset.SSI_target_pool==1)
random.seed()

for group in ['Under 18','18-64', '65 or older']:
    for FIPS in admin.Fips:
        
        if d[str(group)][FIPS] < 0:
            continue
        else:
            this_state = (CPS_dataset.gestfips==FIPS)
            age_range = category[group]
            
            not_imputed = (CPS_dataset.impute==0)
            
            pool_index = CPS_dataset[this_state&age_range&non_current&not_imputed&targets].index
            if len(pool_index)==0:
                print ('there is nothing we could do you this sate', FIPS, 'at age range', group)
                continue
            
            pool = DataFrame({'weight': CPS_dataset.marsupwt[pool_index], 'prob': probs[pool_index]},
                            index=pool_index)
            
            pool = pool.sort_values(by='prob', ascending=False)
            pool['cumsum_weight'] = pool['weight'].cumsum()
            pool['distance'] = abs(pool.cumsum_weight-d[str(group)][FIPS])
            min_index = pool.sort_values(by='distance')[:1].index
            min_weight = int(pool.loc[min_index].cumsum_weight)
            pool['impute'] = np.where(pool.cumsum_weight<=min_weight+10 , 1, 0)
            CPS_dataset.loc[pool.index[pool['impute']==1], 'impute'] = 1
            CPS_dataset.loc[pool.index[pool['impute']==1], 'ssi_impute'] = admin[str(group)+' mean'][FIPS] * 12
        
        print ('we need to impute', d[group][FIPS], 'for state', FIPS, 'In the end we get', 
                CPS_dataset.marsupwt[(CPS_dataset.impute==1)&this_state&age_range].sum())   

('there is nothing we could do you this sate', 1, 'at age range', 'Under 18')
('there is nothing we could do you this sate', 2, 'at age range', 'Under 18')
('we need to impute', 22625.233256351039, 'for state', 4, 'In the end we get', 1129.31)
('we need to impute', 24828.54875288684, 'for state', 5, 'In the end we get', 684.95)
('we need to impute', 107522.95448036952, 'for state', 6, 'In the end we get', 10653.470000000001)
('we need to impute', 7001.3427020785221, 'for state', 8, 'In the end we get', 998.06)
('we need to impute', 7292.259099307159, 'for state', 9, 'In the end we get', 2896.2200000000003)
('there is nothing we could do you this sate', 10, 'at age range', 'Under 18')
('we need to impute', 3914.0509930715934, 'for state', 11, 'In the end we get', 141.49)
('we need to impute', 107371.20337182449, 'for state', 12, 'In the end we get', 8634.16)
('there is nothing we could do you this sate', 13, 'at age range', 'Under 18')
('there is nothing we could do you this sate', 15, 

In [33]:
results = {}

imputed = (CPS_dataset.impute == 1)
has_val = (CPS_dataset.ssi_val != 0)
no_val = (CPS_dataset.ssi_val == 0)

for FIPS in admin.Fips:
    this_state = (CPS_dataset.gestfips==FIPS)
    
    # sum up current recipients total and imputed totals
    current_total = (CPS_dataset.ssi_val*CPS_dataset.marsupwt)[this_state&(CPS_dataset.fwsval<57500)].sum()
    imputed_total = (CPS_dataset.ssi_impute*CPS_dataset.marsupwt)[this_state&imputed].sum()
    on_file = (current_total + imputed_total)/1000000
    
    # w
    federal_total = admin['State Total'][FIPS] * 55000000.0/admin['State Total'].sum()/1000
    state_state = admin["State-administered Supplement 2010"][FIPS] * 721/674/1000000
    admin_total = federal_total + state_state
    
    adjust_ratio = admin_total / on_file
    this_state_num = [admin['State'][FIPS], on_file, admin_total, adjust_ratio]
    results[FIPS] = this_state_num
    

    CPS_dataset.ssi_impute = np.where(has_val&this_state, CPS_dataset.ssi_val * adjust_ratio, CPS_dataset.ssi_impute)
    CPS_dataset.ssi_impute = np.where(no_val&this_state, CPS_dataset.ssi_impute * adjust_ratio, CPS_dataset.ssi_impute)

CPS_dataset["ssi_participation"] = np.zeros(len(CPS_dataset))
CPS_dataset.ssi_participation = np.where(CPS_dataset.impute==1, 2, 0)
CPS_dataset.ssi_participation = np.where(current&(CPS_dataset.fwsval<57500), 1, CPS_dataset.ssi_participation)
CPS_dataset.ssi_impute = np.where(CPS_dataset.ssi_participation==0, 0, CPS_dataset.ssi_impute)

r = DataFrame(results).transpose()
r.columns=['State', 'Imputed', 'Admin', 'adjust ratio']
r.to_csv('amount.csv', index=False)

In [34]:
(CPS_dataset.marsupwt * CPS_dataset.ssi_impute)[CPS_dataset.ssi_participation!=0].sum()

59585000000.00008

In [35]:
CPS_dataset.marsupwt[CPS_dataset.ssi_participation!=0].sum()

8321542.850000003

In [36]:
CPS_dataset.marsupwt[(CPS_dataset.current_recipient==1)&(CPS_dataset.earned==0)].sum()

6345993.43

In [37]:
CPS_dataset.to_csv('SSI_Imputation.csv', 
                   columns=['peridnum', 'ssi_participation', 'ssi_impute', 'h_seq', 'wage', 'a_age', 'fwsval','marsupwt', 'probs'],
                   index=False)
